In [13]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import datetime
#import shapefile
from shapely.ops import snap

root_dir=os.path.dirname(os.getcwd())

brondataversie='20230404'

dijkring='34'
checkbuffer=[0.5,5]
outputversie= datetime.today().strftime("%Y%m%d")

path_shape=os.path.join(root_dir,'output',brondataversie,'brondata','dijkring' + dijkring)

#export path
path_export = os.path.join(root_dir, "output/Network_test",outputversie)
if not os.path.exists(path_export):
    os.makedirs(path_export)


waterloop = gpd.read_file(os.path.join(path_shape,"hydroobject.gpkg"))

for index,row in waterloop.iterrows():
    
    start_point=gpd.points_from_xy([row.geometry.coords.xy[0][0]],[row.geometry.coords.xy[1][0]])
    end_point=gpd.points_from_xy([row.geometry.coords.xy[0][-1]],[row.geometry.coords.xy[1][-1]])
    I_1=waterloop.sindex.query(start_point, predicate="intersects")
    I_2=waterloop.sindex.query(end_point, predicate="intersects")
    I_self=[]
    I_conn_s=[]
    I_conn_e=[]
    str_paralel=''
    str_start=''
    str_end=''

    for check_i in I_1[1]:
      
        if check_i in I_2[1]:

            I_self.append(check_i)
        else:
            target_start=gpd.points_from_xy([waterloop.loc[check_i,'geometry'].coords.xy[0][0]],[waterloop.loc[check_i,'geometry'].coords.xy[1][0]])
            target_end=gpd.points_from_xy([waterloop.loc[check_i,'geometry'].coords.xy[0][-1]],[waterloop.loc[check_i,'geometry'].coords.xy[1][-1]])
            
            if start_point==target_start or start_point==target_end:
                I_conn_s.append(check_i)
    
    for check_i in I_2[1]:
        if check_i not in I_1[1]:
            target_start=gpd.points_from_xy([waterloop.loc[check_i,'geometry'].coords.xy[0][0]],[waterloop.loc[check_i,'geometry'].coords.xy[1][0]])
            target_end=gpd.points_from_xy([waterloop.loc[check_i,'geometry'].coords.xy[0][-1]],[waterloop.loc[check_i,'geometry'].coords.xy[1][-1]])
            
            if end_point==target_start or end_point==target_end:
                I_conn_e.append(check_i)
    
    if len(I_conn_s)==0:

        start_buffer=start_point.buffer(checkbuffer[0])
        i_temp=waterloop.sindex.query(start_buffer, predicate="intersects")
        i_potential=[]
        for check_i in i_temp[1]:
            if check_i not in I_self:
                i_potential.append(check_i)
        if len(i_potential)==0:
            start_buffer=start_point.buffer(checkbuffer[1])
            i_temp=waterloop.sindex.query(start_buffer, predicate="intersects")
            i_potential=[]
            for check_i in i_temp[1]:
                if check_i not in I_self:
                    i_potential.append(check_i)
            if len(i_potential)==0:
                str_start='start point, '
            else:
                target_start=gpd.points_from_xy([waterloop.loc[i,'geometry'].coords.xy[0][0] for i in i_potential],[waterloop.loc[i,'geometry'].coords.xy[1][0] for i in i_potential])
                target_end=gpd.points_from_xy([waterloop.loc[i,'geometry'].coords.xy[0][-1] for i in i_potential],[waterloop.loc[i,'geometry'].coords.xy[1][-1] for i in i_potential])
                all_targets=gpd.GeoDataFrame(geometry=np.append(target_start,target_end))
                all_targets['distance']=start_point.distance(all_targets)
                i_min=all_targets['distance'].idxmin()

                if all_targets.loc[i_min,'distance']<=checkbuffer[1]:
                    waterloop.loc[index,'geometry']=snap(waterloop.loc[index,'geometry'],all_targets.loc[i_min,'geometry'],checkbuffer[1])
                    str_start=f'start punt verplaatst naar punt binnen {checkbuffer[1]}, '
                else:
                    str_start='geen punt in de buurt start, split doel waterloop, '
                # print(all_targets)
                # print(start_point.distance(all_targets).sort)
                


        else:
            target_start=gpd.points_from_xy([waterloop.loc[i,'geometry'].coords.xy[0][0] for i in i_potential],[waterloop.loc[i,'geometry'].coords.xy[1][0] for i in i_potential])
            target_end=gpd.points_from_xy([waterloop.loc[i,'geometry'].coords.xy[0][-1] for i in i_potential],[waterloop.loc[i,'geometry'].coords.xy[1][-1] for i in i_potential])
            all_targets=gpd.GeoDataFrame(geometry=np.append(target_start,target_end))
            all_targets['distance']=start_point.distance(all_targets)
            i_min=all_targets['distance'].idxmin()
 
            if all_targets.loc[i_min,'distance']<=checkbuffer[1]:

                waterloop.loc[index,'geometry']=snap(waterloop.loc[index,'geometry'],all_targets.loc[i_min,'geometry'],checkbuffer[1])
                str_start=f'start punt verplaatst naar punt binnen {checkbuffer[0]}, '
            else:
                str_start='geen punt in de buurt start, split doel waterloop, '        

    if len(I_conn_e)==0:
        
        end_buffer=end_point.buffer(checkbuffer[0])
        i_temp=waterloop.sindex.query(end_buffer, predicate="intersects")
        i_potential=[]
        for check_i in i_temp[1]:
            if check_i not in I_self:
                i_potential.append(check_i)
        if len(i_potential)==0:
            end_buffer=end_point.buffer(checkbuffer[1])
            i_temp=waterloop.sindex.query(end_buffer, predicate="intersects")
            i_potential=[]
            for check_i in i_temp[1]:
                if check_i not in I_self:
                    i_potential.append(check_i)
            if len(i_potential)==0:
                str_end='eind punt. '
            else:
                
                target_start=gpd.points_from_xy([waterloop.loc[i,'geometry'].coords.xy[0][0] for i in i_potential],[waterloop.loc[i,'geometry'].coords.xy[1][0] for i in i_potential])
                target_end=gpd.points_from_xy([waterloop.loc[i,'geometry'].coords.xy[0][-1] for i in i_potential],[waterloop.loc[i,'geometry'].coords.xy[1][-1] for i in i_potential])
                all_targets=gpd.GeoDataFrame(geometry=np.append(target_start,target_end))
                all_targets['distance']=end_point.distance(all_targets)
                i_min=all_targets['distance'].idxmin()

                if all_targets.loc[i_min,'distance']<=checkbuffer[1]:
                    waterloop.loc[index,'geometry']=snap(waterloop.loc[index,'geometry'],all_targets.loc[i_min,'geometry'],checkbuffer[1])
                
                    str_end=f'eind punt verplaatst naar punt binnen {checkbuffer[1]}.'
                else:
                    str_end='geen punt in de buurt einde, split doel waterloop.'
              


        else:
            
            target_start=gpd.points_from_xy([waterloop.loc[i,'geometry'].coords.xy[0][0] for i in i_potential],[waterloop.loc[i,'geometry'].coords.xy[1][0] for i in i_potential])
            target_end=gpd.points_from_xy([waterloop.loc[i,'geometry'].coords.xy[0][-1] for i in i_potential],[waterloop.loc[i,'geometry'].coords.xy[1][-1] for i in i_potential])
            all_targets=gpd.GeoDataFrame(geometry=np.append(target_start,target_end))
            all_targets['distance']=end_point.distance(all_targets)
            i_min=all_targets['distance'].idxmin()
            if all_targets.loc[i_min,'distance']<=checkbuffer[1]:
                waterloop.loc[index,'geometry']=snap(waterloop.loc[index,'geometry'],all_targets.loc[i_min,'geometry'],checkbuffer[1])
                str_end=f'eind punt verplaatst naar punt binnen {checkbuffer[0]}.'
            else:
                str_end='geen punt in de buurt einde, split doel waterloop.' 
    
    
    waterloop.loc[index,'commentconnect']=str_paralel + str_start + str_end


waterloop.to_file(os.path.join(path_export,'network_' + dijkring + '.gpkg'), driver='GPKG')        
print('ready')


ready


In [6]:
help("networkx.readwrite")


Help on package networkx.readwrite in networkx:

NAME
    networkx.readwrite - A package for reading and writing graphs in various formats.

PACKAGE CONTENTS
    adjlist
    edgelist
    gexf
    gml
    gpickle
    graph6
    graphml
    json_graph (package)
    leda
    multiline_adjlist
    nx_shp
    nx_yaml
    p2g
    pajek
    sparse6
    tests (package)
    text

SUBMODULES
    adjacency
    cytoscape
    jit
    node_link
    tree

FUNCTIONS
    __getattr__(name)
        Remove functions and provide informative error messages.

FILE
    c:\users\920307\anaconda3\envs\wbd\lib\site-packages\networkx\readwrite\__init__.py


